In [1]:
import click
import pickle
import numpy as np
import pandas as pd

from features import *
from preprocess_align import *

from quantiprot.utils.io import load_fasta_file
from quantiprot.metrics.aaindex import get_aaindex_file
from quantiprot.utils.feature import Feature, FeatureSet
from quantiprot.utils.sequence import SequenceSet, subset, columns

Using TensorFlow backend.


In [2]:
data_repo = '/home/vedang/Documents/revised_data'

# Retrieve ids set
with open(data_repo + '/orders') as f:
    virus_order, mouse_order, inv_virus_order, inv_mouse_order = pickle.load(f)
    ids_set_virus = set(virus_order.keys())
    ids_set_mouse = set(mouse_order.keys())

with open(data_repo + '/viruses_dict') as f:
    viruses_dict = pickle.load(f)

In [3]:
viruses_dict

{'ASSEM0001': 'A.BrevigMission.1.1918.H1N1.1',
 'ASSEM0002': 'A.Shanghai.4664T.2013.H7N9.1',
 'ASSEM0003': 'A.HongKong.156.1997.H5N1.1',
 'ASSEM0004': 'A.HongKong.156.1997.H5N1.2',
 'ASSEM0005': 'A.PR8F.1934.H1N1.1',
 'ASSEM0006': 'A.PR8M.1934.H1N1.1',
 'ASSEM0007': 'A.Hamburg.05.2009.H1N1.1',
 'ASSEM0008': 'A.Hamburg.NY1580.2009.H1N1.1',
 'ASSEM0009': 'A.Bellamy.1942.H1N1.1',
 'ASSEM0010': 'A.HongKong.1.1968.H3N2.3',
 'ASSEM0011': 'B.Victoria.2.1987.1',
 'ASSEM0012': 'B.Victoria.2.1987.2',
 'ASSEM0013': 'A.swine.Spain.54008.2004.H3N2.1',
 'ASSEM0014': 'A.ma452-G1-1.2014.H5N8.1',
 'ASSEM0015': 'A.ma452-G3-1.2014.H5N8.1',
 'ASSEM0016': 'A.ma452-G3-2.2014.H5N8.1',
 'ASSEM0017': 'A.ma452-G4-1.2014.H5N8.1',
 'ASSEM0018': 'A.ma468-G1-1.2014.H5N8.1',
 'ASSEM0019': 'A.ma468-G1-2.2014.H5N8.1',
 'ASSEM0020': 'A.ma468-G2-1.2014.H5N8.1',
 'ASSEM0021': 'A.ma468-G2-2.2014.H5N8.1',
 'ASSEM0022': 'A.ma468-G2-3.2014.H5N8.1',
 'ASSEM0023': 'A.ma468-G4-2.2014.H5N8.1',
 'ASSEM0024': 'A.swine.Sweden.1021.

In [4]:
f = load_fasta_file('data/virus/Seg2p1_prep.fasta')

Quantiprot SequenceSet object
  'name': data/virus/Seg2p1_prep.fasta
  'unique': True
  number of sequences: 330
